# Import all dependencies

In [ ]:

import datetime

from app import crud, models, schemas
from app.api import deps
from app.worker import FetchUIPathToken, GetUIPathToken

from app.core.uipapiconfig import (
    uipclient_folders,
    uipclient_jobs,
    uipclient_queueuitems,
    uipclient_queueuitemevents,
    uipclient_queuedefinitions,
    uipclient_processes,
    uipclient_sessions,
)

from app.db.session import SessionLocal
db = SessionLocal()
print(db)

In [ ]:
# Get Auth token just in case

FetchUIPathToken()

# Folders

In [32]:
select = schemas.FolderGETResponse.get_select_filter()
print(select)
folders = uipclient_folders.folders_get(select=select)
folderlist = [schemas.FolderGETResponse.parse_from_swagger(folder.to_dict(), folder.attribute_map) for folder in folders.value]
for folder in folderlist:
    crud.uip_folder.create_safe(db=db, obj_in=folder)
print(folderlist)

Key, DisplayName, FullyQualifiedName, Description, FolderType, ParentId, ParentKey, Id


/app/app/crud/base.py:40: SAWarning: New instance <Folder at 0x7fcf723b8110> with identity key (<class 'app.models.orchestratorapi.Folder'>, (2440043,), None) conflicts with persistent instance <Folder at 0x7fcf72c08210>
  db.commit()


[FolderGETResponse(Key=UUID('b4f13f74-7060-4b11-8729-deaf0a6bf206'), DisplayName='Shared', FullyQualifiedName='Shared', Description=None, FolderType='Standard', ParentId=None, ParentKey=None, Id=2440043), FolderGETResponse(Key=UUID('d7bfe307-62a5-48e1-ba5f-9c7adc6bfda7'), DisplayName='TestFolder1', FullyQualifiedName='TestFolder1', Description=None, FolderType='Standard', ParentId=None, ParentKey=None, Id=4572437), FolderGETResponse(Key=UUID('9ff0f0a9-27e4-48eb-8845-033e4a6e333e'), DisplayName='TestFolder2', FullyQualifiedName='TestFolder2', Description=None, FolderType='Standard', ParentId=None, ParentKey=None, Id=4572438), FolderGETResponse(Key=UUID('094752d6-d1bf-4540-bc37-f62f3a5b72c2'), DisplayName='TestSubFolder1', FullyQualifiedName='TestFolder1/TestSubFolder1', Description=None, FolderType='Standard', ParentId=4572437, ParentKey=UUID('d7bfe307-62a5-48e1-ba5f-9c7adc6bfda7'), Id=4572440), FolderGETResponse(Key=UUID('07c5eff7-d971-4b45-8f8c-bc81c55096b5'), DisplayName='TestSubfold

In [30]:
#CHECK FOLDER 

folderlist = [2440043]

for folderid in folderlist:
    a = crud.uip_folder.get(db=db, id=folderid)

# Jobs

In [ ]:
select = schemas.JobGETResponseExtended.get_select_filter()
lastgettime = datetime.datetime(2023,7,28,10,13,2)
filter=f"CreationTime gt {lastgettime.isoformat()}Z"
print(filter)
print(select)
for folder in folderlist:
    x_uipath_organization_unit_id = folder
    jobs = uipclient_jobs.jobs_get(select=select, x_uipath_organization_unit_id=x_uipath_organization_unit_id, filter=filter)
    joblist = [schemas.JobGETResponseExtended.parse_from_swagger(job.to_dict(), job.attribute_map) for job in jobs.value]
    for job in joblist:
        crud.uip_job.upsert(db=db, obj_in=job)

# Processes( releases)

In [ ]:
select = schemas.ProcessGETResponse.get_select_filter()
#lastgettime = datetime.datetime(2023,7,28,10,13,2)
#filter=f"CreationTime gt {lastgettime.isoformat()}Z"
#print(filter)
#print(select)
for folder in folderlist:
    x_uipath_organization_unit_id = folder.Id
    processes = uipclient_processes.releases_get(x_uipath_organization_unit_id=x_uipath_organization_unit_id)
    processes = [schemas.ProcessGETResponse.parse_from_swagger(process.to_dict(),  process.attribute_map) for process in processes.value]
    for process in processes:
        crud.uip_process.create_safe(db=db, obj_in=process)
    

# QueueDefinitions

In [ ]:
select = schemas.QueueDefinitionGETResponse.get_select_filter()
#lastgettime = datetime.datetime(2023,7,28,10,13,2)
#filter=f"CreationTime gt {lastgettime.isoformat()}Z"
#print(filter)
#print(select)
for folder in folderlist:
    x_uipath_organization_unit_id = folder.Id
    queues = uipclient_queuedefinitions.queue_definitions_get(x_uipath_organization_unit_id=x_uipath_organization_unit_id)
    queues = [schemas.QueueDefinitionGETResponse.parse_from_swagger(queue.to_dict(),  queue.attribute_map) for queue in queues.value]
    for queue in queues:
        crud.uip_queue_definitions.create_safe(db=db, obj_in=queue)
    

# Queue Items

In [ ]:
select = schemas.QueueItemGETResponse.get_select_filter()
#lastgettime = datetime.datetime(2023,7,28,10,13,2)
#filter=f"CreationTime gt {lastgettime.isoformat()}Z"
#print(filter)
print(select)
for folder in folderlist:
    #TODO: Get list of qdefinitionID
    x_uipath_organization_unit_id = folder.Id
    #x_uipath_organization_unit_id=x_uipath_organization_unit_id
    qitems = uipclient_queueuitems.queue_items_get(x_uipath_organization_unit_id=x_uipath_organization_unit_id)
    qitems = [schemas.QueueItemGETResponse.parse_from_swagger(qitem.to_dict(),  qitem.attribute_map) for qitem in qitems.value]
    for qitem in qitems:
        crud.uip_queue_item.upsert(db=db, obj_in=qitem)
    

# Queue Item Events

In [38]:
select = schemas.QueueItemEventGETResponse.get_select_filter()
#lastgettime = datetime.datetime(2023,7,28,10,13,2)
#filter=f"CreationTime gt {lastgettime.isoformat()}Z"
#print(filter)
#print(select)
for folder in folderlist:
    print(folder.Id)
    x_uipath_organization_unit_id = folder.Id
    #x_uipath_organization_unit_id=x_uipath_organization_unit_id
    qitemevents = uipclient_queueuitemevents.queue_item_events_get(x_uipath_organization_unit_id=x_uipath_organization_unit_id)
    qitemevents = [schemas.QueueItemEventGETResponse.parse_from_swagger(event.to_dict(),  event.attribute_map) for event in qitemevents.value]
    for event in qitemevents:
        #crud.uip_queue_item_event.create_safe(db=db, obj_in=event)
        ...
    

2440043
4572437
4572438
4572440
4572441
4602674


In [50]:

qitemevents = uipclient_queueuitemevents.queue_item_events_get(x_uipath_organization_unit_id=4572437)
qitemeventsPydantic = [schemas.QueueItemEventGETResponse.parse_from_swagger(event.to_dict(),  event.attribute_map) for event in qitemevents.value]



QueueItemId=422372035 Timestamp=datetime.datetime(2024, 3, 27, 12, 6, 22, 980000, tzinfo=tzlocal()) Action='Edit' Status='Failed' Id=648916430


In [51]:
print(qitemevents)

{'value': [{'action': 'Edit',
            'data': None,
            'external_client_id': None,
            'id': 648916430,
            'queue_item_id': 422372035,
            'review_status': 'None',
            'reviewer_user_id': None,
            'reviewer_user_name': None,
            'status': 'Failed',
            'timestamp': datetime.datetime(2024, 3, 27, 12, 6, 22, 980000, tzinfo=tzlocal()),
            'user_id': 3661046,
            'user_name': 'jorge42caravaca@gmail.com'},
           {'action': 'Status',
            'data': None,
            'external_client_id': None,
            'id': 648916424,
            'queue_item_id': 422372035,
            'review_status': 'None',
            'reviewer_user_id': None,
            'reviewer_user_name': None,
            'status': 'InProgress',
            'timestamp': datetime.datetime(2024, 3, 27, 12, 6, 21, 667000, tzinfo=tzlocal()),
            'user_id': 3661046,
            'user_name': 'jorge42caravaca@gmail.com'},
        

In [52]:
print(qitemeventsPydantic[0])

QueueItemId=422372035 Timestamp=datetime.datetime(2024, 3, 27, 12, 6, 22, 980000, tzinfo=tzlocal()) Action='Edit' Status='Failed' Id=648916430


In [59]:
stringtoparse="2024-03-27T12:06:22.98Z"
from dateutil.parser import parse
from datetime import timezone
parsed=parse(stringtoparse,)
othertz = parsed.astimezone(tz=timezone.utc)
parsed


datetime.datetime(2024, 3, 27, 12, 6, 22, 980000, tzinfo=tzlocal())

In [60]:
othertz

datetime.datetime(2024, 3, 27, 12, 6, 22, 980000, tzinfo=datetime.timezone.utc)

# Sessions

In [ ]:
select = schemas.SessionGETResponse.get_select_filter()
#lastgettime = datetime.datetime(2023,7,28,10,13,2)
#filter=f"CreationTime gt {lastgettime.isoformat()}Z"
#print(filter)
#print(select)
runtime_type="Unattended"
for folder in folderlist:
    x_uipath_organization_unit_id = folder.Id
    #x_uipath_organization_unit_id=x_uipath_organization_unit_id
    #uipclient_sessions.sessions_get_machine_session_runtimes
    sessions = uipclient_sessions.sessions_get_machine_session_runtimes(runtime_type=runtime_type)
    sessions = [schemas.SessionGETResponse.parse_from_swagger(session.to_dict(),  session.attribute_map) for session in sessions.value]
    for session in sessions:
        print(session)
        crud.uip_session.upsert(db=db, obj_in=session)
    